# plugin manager

> Plugin discovery, loading, and lifecycle management system

In [ ]:
#| default_exp plugin_manager

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import importlib.metadata
import inspect
import logging
from pathlib import Path
import sys
from typing import Dict, List, Optional, Type, Any
from cjm_transcription_plugin_system.core import AudioData, TranscriptionResult
from cjm_transcription_plugin_system.plugin_interface import PluginInterface, PluginMeta

In [ ]:
#| export
class PluginManager:
    """Manages plugin discovery, loading, and lifecycle."""
    
    def __init__(self, 
                 plugin_interface: Type[PluginInterface] = PluginInterface,  # The base class/interface plugins must implement
                 entry_point_group: str = "transcription.plugins"  # The entry point group name for plugin discovery
                ):
        """
        Initialize the plugin manager.
        """
        self.plugin_interface = plugin_interface
        self.entry_point_group = entry_point_group
        self.plugins: Dict[str, PluginMeta] = {}
        self._loaded_modules: Dict[str, Any] = {}
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
    
    def discover_plugins(
        self
    ) -> List[PluginMeta]:  # List of discovered plugin metadata objects
        """
        Discover all installed plugins via entry points.
        
        This method looks for plugins installed as packages that declare
        entry points in the specified group.
        """
        discovered = []
        
        try:
            # Python 3.10+ and backported to 3.8+
            entry_points = importlib.metadata.entry_points()
            if hasattr(entry_points, 'select'):
                # Python 3.10+
                plugin_eps = entry_points.select(group=self.entry_point_group)
            else:
                # Python 3.8-3.9
                plugin_eps = entry_points.get(self.entry_point_group, [])
        except AttributeError:
            # Fallback for older Python versions
            import pkg_resources
            plugin_eps = pkg_resources.iter_entry_points(self.entry_point_group)
        
        for ep in plugin_eps:
            try:
                # Get package metadata
                if hasattr(ep, 'dist'):
                    # pkg_resources style
                    dist = ep.dist
                    version = dist.version
                    package_name = dist.project_name
                else:
                    # importlib.metadata style
                    package_name = ep.name
                    try:
                        dist = importlib.metadata.distribution(ep.dist.name if hasattr(ep, 'dist') else ep.name)
                        version = dist.version
                    except:
                        version = "unknown"
                
                meta = PluginMeta(
                    name=ep.name,
                    version=version,
                    package_name=package_name
                )
                discovered.append(meta)
                self.logger.info(f"Discovered plugin: {meta.name} v{meta.version}")
                
            except Exception as e:
                self.logger.error(f"Error discovering plugin {ep.name}: {e}")
        
        return discovered
    
    def load_plugin(
        self,
        plugin_meta: PluginMeta,  # The plugin metadata
        config: Optional[Dict[str, Any]] = None  # Optional configuration for the plugin
    ) -> bool:  # True if successfully loaded, False otherwise
        """
        Load and initialize a plugin.
        """
        try:
            # Find the entry point and load it
            entry_points = importlib.metadata.entry_points()
            if hasattr(entry_points, 'select'):
                plugin_eps = entry_points.select(group=self.entry_point_group, name=plugin_meta.name)
            else:
                plugin_eps = [ep for ep in entry_points.get(self.entry_point_group, []) 
                             if ep.name == plugin_meta.name]
            
            if not plugin_eps:
                self.logger.error(f"Plugin {plugin_meta.name} not found in entry points")
                return False
            
            ep = list(plugin_eps)[0]
            plugin_class = ep.load()
            
            # Verify it implements the required interface
            if not issubclass(plugin_class, self.plugin_interface):
                self.logger.error(f"Plugin {plugin_meta.name} does not implement required interface")
                return False
            
            # Instantiate and initialize the plugin
            plugin_instance = plugin_class()
            plugin_instance.initialize(config)
            
            # Store the plugin
            plugin_meta.instance = plugin_instance
            self.plugins[plugin_meta.name] = plugin_meta
            
            self.logger.info(f"Loaded plugin: {plugin_meta.name}")
            return True
            
        except Exception as e:
            self.logger.error(f"Error loading plugin {plugin_meta.name}: {e}")
            return False
    
    def load_plugin_from_module(
        self,
        module_path: str,  # Path to the Python module
        config: Optional[Dict[str, Any]] = None  # Optional configuration for the plugin
    ) -> bool:  # True if successfully loaded, False otherwise
        """
        Load a plugin directly from a Python module file or package.
        Useful for development or local plugins.
        """
        try:
            # Convert to Path object
            path = Path(module_path)
            
            # Load the module
            if path.is_file():
                # Single file module
                spec = importlib.util.spec_from_file_location(path.stem, path)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)
            else:
                # Package directory
                sys.path.insert(0, str(path.parent))
                module = importlib.import_module(path.name)
            
            # Find plugin classes in the module
            plugin_classes = []
            for name, obj in inspect.getmembers(module):
                if (inspect.isclass(obj) and 
                    issubclass(obj, self.plugin_interface) and 
                    obj != self.plugin_interface):
                    plugin_classes.append(obj)
            
            if not plugin_classes:
                self.logger.error(f"No plugin classes found in {module_path}")
                return False
            
            # Load the first plugin class found
            plugin_class = plugin_classes[0]
            plugin_instance = plugin_class()
            plugin_instance.initialize(config)
            
            # Create metadata
            meta = PluginMeta(
                name=plugin_instance.name,
                version=plugin_instance.version,
                package_name=str(module_path)
            )
            meta.instance = plugin_instance
            
            self.plugins[meta.name] = meta
            self._loaded_modules[meta.name] = module
            
            self.logger.info(f"Loaded plugin from module: {meta.name}")
            return True
            
        except Exception as e:
            self.logger.error(f"Error loading plugin from {module_path}: {e}")
            return False
    
    def unload_plugin(
        self,
        plugin_name: str  # Name of the plugin to unload
    ) -> bool:  # True if successfully unloaded, False otherwise
        """
        Unload a plugin and call its cleanup method.
        """
        if plugin_name not in self.plugins:
            self.logger.error(f"Plugin {plugin_name} not found")
            return False
        
        try:
            plugin_meta = self.plugins[plugin_name]
            if plugin_meta.instance:
                plugin_meta.instance.cleanup()
            
            del self.plugins[plugin_name]
            
            # Remove from loaded modules if it was loaded that way
            if plugin_name in self._loaded_modules:
                del self._loaded_modules[plugin_name]
            
            self.logger.info(f"Unloaded plugin: {plugin_name}")
            return True
            
        except Exception as e:
            self.logger.error(f"Error unloading plugin {plugin_name}: {e}")
            return False
    
    def get_plugin(
        self,
        plugin_name: str  # The name of the plugin to retrieve
    ) -> Optional[PluginInterface]:  # The plugin instance if found, None otherwise
        """Get a loaded plugin instance by name."""
        if plugin_name in self.plugins:
            return self.plugins[plugin_name].instance
        return None
    
    def list_plugins(
        self
    ) -> List[PluginMeta]:  # List of metadata for all loaded plugins
        """List all loaded plugins."""
        return list(self.plugins.values())
    
    def execute_plugin(
        self,
        plugin_name: str,  # Name of the plugin to execute
        *args,  # Arguments to pass to the plugin
        **kwargs  # Key word arguments to pass to the plugin
    ) -> Any:  # The result of the plugin execution
        """
        Execute a plugin's main functionality.            
        """
        plugin = self.get_plugin(plugin_name)
        if not plugin:
            raise ValueError(f"Plugin {plugin_name} not found or not loaded")
        
        if not self.plugins[plugin_name].enabled:
            raise ValueError(f"Plugin {plugin_name} is disabled")
        
        return plugin.execute(*args, **kwargs)
    
    def enable_plugin(
        self,
        plugin_name: str  # The name of the plugin to enable
    ) -> bool:  # True if plugin was enabled, False if not found
        """Enable a plugin."""
        if plugin_name in self.plugins:
            self.plugins[plugin_name].enabled = True
            return True
        return False
    
    def disable_plugin(
        self,
        plugin_name: str  # The name of the plugin to disable
    ) -> bool:  # True if plugin was disabled, False if not found
        """Disable a plugin without unloading it."""
        if plugin_name in self.plugins:
            self.plugins[plugin_name].enabled = False
            return True
        return False

In [ ]:
#| eval: false
logging.basicConfig(level=logging.INFO)
from nbdev.config import get_config
project_config = get_config()

test_files_dir = project_config.config_path/"test_files"
file_path = test_files_dir/"example_plugin.py"

# Create plugin manager
plugin_manager = PluginManager()

# Discover installed plugins
print("Discovering plugins...")
discovered = plugin_manager.discover_plugins()
print(f"Found {len(discovered)} plugins")

# Load discovered plugins
for plugin_meta in discovered:
    plugin_manager.load_plugin(plugin_meta, config={"debug": True})

print(plugin_manager.load_plugin_from_module(file_path))
example_plugin_name = plugin_manager.list_plugins()[0].name
print(f"Plugin Name: {example_plugin_name}")

# List loaded plugins
print("\nLoaded plugins:")
for meta in plugin_manager.list_plugins():
    print(f"  - {meta.name} v{meta.version} (enabled: {meta.enabled})")

# Execute a plugin
if plugin_manager.get_plugin(example_plugin_name):
    result = plugin_manager.execute_plugin(example_plugin_name, "test", key="value")
    print(f"\nPlugin result: {result}")

# Disable/enable plugins
plugin_manager.disable_plugin(example_plugin_name)
plugin_manager.enable_plugin(example_plugin_name)

INFO:example_plugin.ExamplePlugin:Initializing example_plugin with config: {}
INFO:__main__.PluginManager:Loaded plugin from module: example_plugin
INFO:example_plugin.ExamplePlugin:Example plugin executed with args: ('test',), kwargs: {'key': 'value'}


Discovering plugins...
Found 0 plugins
True
Plugin Name: example_plugin

Loaded plugins:
  - example_plugin v1.0.0 (enabled: True)

Plugin result: This is an example transcription.


True

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()